# Universal Sets

**Universal sets** are symbolic constructs in DataJoint that represent the set of all possible values for specified attributes. They enable queries that extract unique values or perform aggregations without a natural grouping entity.

## Understanding Universal Sets

The `dj.U()` class creates a universal set—a conceptual table that can be restricted or used in aggregations. Universal sets are not directly fetchable; they serve as operands in query expressions.

### Two Forms

| Form | Meaning | Primary Key |
|------|---------|-------------|
| `dj.U('attr1', 'attr2', ...)` | All possible combinations of the specified attributes | The specified attributes |
| `dj.U()` | A singular universal entity (one conceptual row) | Empty set |

## Extracting Unique Values

The most common use of universal sets is extracting distinct values from a table.

### Basic Syntax

```python
# Get unique values of an attribute
unique_values = dj.U('attribute_name') & SomeTable
```

When restricted by an existing table, `dj.U(<attributes>)` returns the distinct values of those attributes present in the table.

### Example: Unique First Names

```python
# All unique first names among students
unique_first_names = dj.U('first_name') & Student
```

**SQL Equivalent**:
```sql
SELECT DISTINCT first_name FROM student;
```

### Example: Unique Name Combinations

```python
# All unique first_name + last_name combinations
unique_full_names = dj.U('first_name', 'last_name') & Student
```

**SQL Equivalent**:
```sql
SELECT DISTINCT first_name, last_name FROM student;
```

## Restricting Unique Values

Universal sets can be combined with restrictions to find unique values within filtered subsets.

### Example: Unique Names of Male Students

```python
# Unique first names among male students only
male_names = dj.U('first_name') & (Student & {'sex': 'M'})
```

**SQL Equivalent**:
```sql
SELECT DISTINCT first_name FROM student WHERE sex = 'M';
```

### Example: Birth Years of Current Students

```python
# Unique birth years among students enrolled in current term
birth_years = dj.U('year') & (
    Student.proj(year='YEAR(date_of_birth)') & (Enroll & CurrentTerm)
)
```

This extracts the unique birth years from a projection that computes the year from the date of birth.

## Universal Aggregation

The empty universal set `dj.U()` represents a single entity that spans all rows. It's used for aggregations that summarize an entire table rather than grouping by a specific entity.

### Total Count

```python
# Count total number of students
total_count = dj.U().aggr(Student, n_students='COUNT(*)')
```

**Result**: A table with one row and one attribute `n_students` containing the count.

**SQL Equivalent**:
```sql
SELECT COUNT(*) AS n_students FROM student;
```

### Multiple Aggregate Statistics

```python
# Compute multiple statistics across all students
student_stats = dj.U().aggr(
    Student.proj(age='TIMESTAMPDIFF(YEAR, date_of_birth, CURDATE())'),
    n_students='COUNT(*)',
    avg_age='AVG(age)',
    min_age='MIN(age)',
    max_age='MAX(age)'
)
```

**SQL Equivalent**:
```sql
SELECT 
    COUNT(*) AS n_students,
    AVG(TIMESTAMPDIFF(YEAR, date_of_birth, CURDATE())) AS avg_age,
    MIN(TIMESTAMPDIFF(YEAR, date_of_birth, CURDATE())) AS min_age,
    MAX(TIMESTAMPDIFF(YEAR, date_of_birth, CURDATE())) AS max_age
FROM student;
```

## Aggregation by Arbitrary Groupings

When you need to aggregate data by attributes that don't form a natural entity type in your schema, use `dj.U(<attributes>)` to create an arbitrary grouping.

### Example: Students per Birth Year and Month

```python
# Count students born in each year and month
student_counts = dj.U('birth_year', 'birth_month').aggr(
    Student.proj(
        birth_year='YEAR(date_of_birth)', 
        birth_month='MONTH(date_of_birth)'
    ),
    n_students='COUNT(*)'
)
```

**SQL Equivalent**:
```sql
SELECT 
    YEAR(date_of_birth) AS birth_year,
    MONTH(date_of_birth) AS birth_month,
    COUNT(*) AS n_students
FROM student
GROUP BY YEAR(date_of_birth), MONTH(date_of_birth);
```

### Example: Enrollments per Department per Term

```python
# Count enrollments by department and term
enrollment_counts = dj.U('dept', 'term').aggr(
    Enroll * Section,
    n_enrollments='COUNT(*)'
)
```

This creates a grouping by department and term without requiring a DepartmentTerm entity in your schema.

## Examples from the University Database

### Example 1: Unique First Names by Gender

```python
# All unique first names among male students
male_names = dj.U('first_name') & (Student & {'sex': 'M'})

# All unique first names among female students
female_names = dj.U('first_name') & (Student & {'sex': 'F'})
```

### Example 2: Birth Years of Enrolled Students

```python
# Show all birth years for students enrolled in current term
birth_years = dj.U('year') & (
    Student.proj(year='YEAR(date_of_birth)') & (Enroll & CurrentTerm)
)
```

### Example 3: Department Statistics

```python
# Count students in each department
dept_counts = Department.aggr(StudentMajor, count='COUNT(student_id)')

# Count male and female students per department
gender_counts = Department.aggr(
    StudentMajor * Student, 
    males='SUM(sex="M")', 
    females='SUM(sex="F")'
)
```

### Example 4: GPA and Credits Summary

```python
# Overall average GPA across all graded students
overall_gpa = dj.U().aggr(
    Course * Grade * LetterGrade,
    avg_gpa='SUM(points * credits) / SUM(credits)'
)
```

## Comparison: Universal Sets vs. Standard Aggregation

| Approach | Use Case | Result Primary Key |
|----------|----------|-------------------|
| `Entity.aggr(Related, ...)` | Aggregate by existing entity type | Entity's primary key |
| `dj.U('attrs').aggr(Table, ...)` | Aggregate by arbitrary grouping | Specified attributes |
| `dj.U().aggr(Table, ...)` | Universal aggregate (whole table) | Empty (single row) |

### When to Use Each

**Standard aggregation** (`Entity.aggr(...)`):
- When grouping by an existing entity (e.g., count enrollments per student)
- Result represents augmented entities

**Arbitrary grouping** (`dj.U('attrs').aggr(...)`):
- When grouping by computed or non-entity attributes (e.g., count by birth year)
- Result represents a new grouping not in your schema

**Universal aggregate** (`dj.U().aggr(...)`):
- When computing totals across an entire table
- Result is a single row of summary statistics

## SQL Translation

### Unique Values

```python
# DataJoint
dj.U('first_name') & Student
```

```sql
-- SQL
SELECT DISTINCT first_name FROM student;
```

### Universal Aggregation

```python
# DataJoint
dj.U().aggr(Student, count='COUNT(*)')
```

```sql
-- SQL
SELECT COUNT(*) AS count FROM student;
```

### Arbitrary Grouping

```python
# DataJoint
dj.U('home_state').aggr(Student, n='COUNT(*)')
```

```sql
-- SQL
SELECT home_state, COUNT(*) AS n
FROM student
GROUP BY home_state;
```

## Best Practices

### 1. Choose the Right Approach

- For unique values: `dj.U('attr') & Table`
- For total counts/sums: `dj.U().aggr(Table, ...)`
- For grouping by non-entity attributes: `dj.U('attr').aggr(Table, ...)`

### 2. Use Projection to Create Grouping Attributes

When grouping by computed values, project them first:

```python
# Project to create the grouping attribute
with_year = Student.proj(birth_year='YEAR(date_of_birth)')

# Then aggregate by that attribute
counts_by_year = dj.U('birth_year').aggr(with_year, n='COUNT(*)')
```

### 3. Understand the Result Structure

- `dj.U('attr') & Table` — primary key is `attr`
- `dj.U().aggr(...)` — primary key is empty (single-row result)

### 4. Combine with Restrictions for Filtered Results

```python
# Unique departments among currently enrolled students
active_depts = dj.U('dept') & (Enroll & CurrentTerm)
```

## Summary

Universal sets provide three key capabilities:

1. **Extract unique values**: `dj.U('attr') & Table` returns distinct values
2. **Universal aggregation**: `dj.U().aggr(Table, ...)` summarizes entire tables
3. **Arbitrary grouping**: `dj.U('attrs').aggr(Table, ...)` groups by non-entity attributes

Use universal sets when:
- You need distinct values from a table
- You want totals across an entire table
- You need to group by attributes that don't form a natural entity in your schema

## Practice Exercises

### Exercise 1: Unique Values

**Task**: Find all unique home states represented by students.

```python
unique_states = dj.U('home_state') & Student
```

### Exercise 2: Universal Aggregation

**Task**: Count the total number of course enrollments.

```python
total_enrollments = dj.U().aggr(Enroll, n='COUNT(*)')
```

### Exercise 3: Filtered Unique Values

**Task**: Find unique departments that have students enrolled in the current term.

```python
active_depts = dj.U('dept') & (Enroll & CurrentTerm)
```

### Exercise 4: Arbitrary Grouping

**Task**: Count students by home state.

```python
students_by_state = dj.U('home_state').aggr(Student, n_students='COUNT(*)')
```

### Exercise 5: Computed Grouping

**Task**: Count students by birth year.

```python
students_by_year = dj.U('birth_year').aggr(
    Student.proj(birth_year='YEAR(date_of_birth)'),
    n_students='COUNT(*)'
)
```

:::{seealso}
For more examples using universal sets, see the [University Queries](../80-examples/016-university-queries.ipynb) example.
:::